In [8]:
from ipywidgets import *
from IPython.display import display as dsp
from IPython import display
import numpy as np
import matplotlib.pyplot as plt
import time
import seaborn as sns
from envs.MazeGridWorld import MazeGridWorld
from pylab import *
from train import *

def policy(state):
    choices = []
    for action in range(4):
        a = env.next_states(state, action)
        probs = []
        for i in a:
            probs.append(i[1])
        new_state = a[np.argmax(probs)][0]
        choices.append(V[new_state])

    return np.argmax(choices)

def calculate_action_expectation(state, action, Values):
    ps = env.next_states(state, action)
    summation = 0
    for s, p in ps: 
        summation += Values[s] * p
    
    return summation

env = None
%matplotlib inline
plt.rcParams['figure.dpi'] = 150
epochs = 0

@interact(x=widgets.Text(value='40', description = 'Epochs'))
def CreateMaze(x):
    global epochs
    epochs = int(x)

@interact(x=widgets.Text(value='15'), y=widgets.Text(value='15'))
def CreateMaze(x, y):
    global env
    env = MazeGridWorld(grid_size=(int(x), int(y)))
    s_0 = env.reset()
    plt.axis('off')
    env.draw_state()
    
train_button = widgets.Button(
    description='Train',
    disabled=False,
    button_style='warning', 
    tooltip='Train'
)

solve_button = widgets.Button(
    description='Solve',
    disabled=False,
    button_style='info',
    tooltip='Solve'
)
V = None
def on_train_pressed(b):
    global V
    random_policy = lambda state: np.random.randint(4)
    V = np.zeros(env.w*env.h).reshape(env.w,env.h)
    R = np.zeros(env.w*env.h).reshape(env.w,env.h)
    R[env.goal] = 1
    iter_count=epochs
    gamma=0.99
    visual = True
    plt.axis('equal')

    for epoch in range(iter_count):
            for i in range(V.shape[0]):
                for j in range(V.shape[1])[::-1]:
                    if env.field[i, j] != 1:
                        prev = V[i, j]*1
                        V[i, j] = max([R[i, j] + gamma*(calculate_action_expectation((i, j), a, V)) for a in range(4)])
                        if visual and prev != V[i, j]:
                            plt.subplot(1, 2, 1)
                            env.reset()
                            plt.axis('off')
                            env.draw_state()
                            plt.subplot(1, 2, 2)
                            plt.axis('equal')
                            plt.axis('off')
                            sns.heatmap(np.rot90(V), cbar=None)
                            plt.suptitle('Epoch '+str(epoch+1)+'/'+str(iter_count), fontsize=20)
                            display.clear_output(wait=True)
                            display.display(plt.gcf())
                            time.sleep(0.000000000000000000000000000000000000000000000000000001)
                            plt.gcf().clear()

    dsp(train_button, solve_button);
    
def on_solve_pressed(b): 
    global V
    plt.subplot(1, 2, 2)
    plt.axis('equal')
    plt.axis('off')
    sns.heatmap(np.rot90(V), cbar=None)
    plt.subplot(1, 2, 1)
    plt.axis('off')
    reward1 = env.play_with_policy(policy, max_iter=100)
    
train_button.on_click(on_train_pressed)
solve_button.on_click(on_solve_pressed)
dsp(train_button, solve_button);

interactive(children=(Text(value='40', description='Epochs'), Output()), _dom_classes=('widget-interact',))

interactive(children=(Text(value='15', description='x'), Text(value='15', description='y'), Output()), _dom_cl…

Button(button_style='warning', description='Train', style=ButtonStyle(), tooltip='Train')

Button(button_style='info', description='Solve', style=ButtonStyle(), tooltip='Solve')